In [11]:
import numpy as np
import pandas as pd
import scanpy as sc
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import harmonypy
import pymn
import scrublet as scr
import gseapy as gp
from gseapy.plot import barplot, dotplot
import bottleneck
import pyreadr
from scipy.io import mmread
import scipy
from sklearn import preprocessing
import random as rd
import scib
from matplotlib_venn import venn2,venn2_circles,venn3
from pyscenic.export import export2loom, add_scenic_metadata
from sklearn.decomposition import NMF
from sklearn import metrics
import palettable
from pyscenic.rss import regulon_specificity_scores
import matplotlib.image as mpimg
import cosg as cosg

In [12]:
sc.set_figure_params(dpi = 200, color_map = 'viridis_r' )
sc.settings.verbosity = 2

/public/home/guogjgroup/ggj/anaconda3/lib/python3.9/site-packages/scanpy/_settings.py:447: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  IPython.display.set_matplotlib_formats(*ipython_format)


In [3]:
#https://github.com/sqjin/CellChat

In [ ]:
# https://github.com/sqjin/CellChat/issues/529
# https://github.com/sqjin/CellChat/issues/550
# https://github.com/sqjin/CellChat/issues/521

In [ ]:
# https://www.nature.com/articles/s41467-021-21246-9
# Both CellChat and CellPhoneDB were relatively robust to subsampling, which is likely because both methods infer 
# cell–cell communication based on cell clusters. Such robustness in terms of subsampling is very useful when 
# analyzing the rapidly growing volume of scRNA-seq data

In [32]:
def gene_name_in_gtf(query_list):
    
    import numpy as np
    import pandas as pd
    
    gene_alias_latest=pd.read_csv('/public/home/guogjgroup/ggj/matq_analysis/scripts_share/jikai/hh_seq_annotation/gene_alias.csv', index_col=0)
    gene_alias_latest=gene_alias_latest.fillna('0')
    gene_alias_latest['alias']=gene_alias_latest.astype(str)
    
    query_result=pd.DataFrame(index=query_list, columns=['name_in_gtf'])
    query_result['name_in_gtf']='Not_Found'

    for i in query_list:

        if [s for s in gene_alias_latest.index.tolist() if i == s]!=[]:
            query_result.loc[i,'name_in_gtf']=i

        elif [s for s in gene_alias_latest['alias'].tolist() if i in s]==[] and [s for s in gene_alias_latest.index.tolist() if i == s]==[]:
            continue

        elif [s for s in gene_alias_latest['alias'].tolist() if i in s]!=[] and [s for s in gene_alias_latest.index.tolist() if i == s]==[]:
            query_gene_column=[s for s in gene_alias_latest['alias'].tolist() if i in s]
            query_gene_subset=gene_alias_latest[gene_alias_latest['alias'].isin(query_gene_column)]
            query_gene_subset=query_gene_subset['alias'].str.split('|', expand=True)
            query_gene_subset=query_gene_subset.fillna('0')
            if query_gene_subset.shape[0]==1 and [s for s in np.unique(query_gene_subset.values).tolist() if i == s]!=[]:
                query_result.loc[i,'name_in_gtf']=''.join(query_gene_subset.index.tolist())
            elif query_gene_subset.shape[0]>1 and [s for s in np.unique(query_gene_subset.values).tolist() if i == s]!=[]:
                for each_row in query_gene_subset.index.tolist():
                    if [s for s in query_gene_subset.loc[each_row,:].tolist() if i == s]==[]:
                        query_gene_subset=query_gene_subset.drop(each_row)
                if query_gene_subset.shape[0]==0:
                    continue
                elif query_gene_subset.shape[0]>1:
                    query_result.loc[i,'name_in_gtf']='Ambiguous'
                elif query_gene_subset.shape[0]==1:
                    query_result.loc[i,'name_in_gtf']=''.join(query_gene_subset.index.tolist())
    print(query_result)
    query_result.to_csv('query_result.csv')

# Gene Name

In [88]:
cellchat_gene_name = np.unique(pd.read_csv("cell_chat_gene_name.csv", index_col=0)['x']).tolist()

In [35]:
gene_name_in_gtf(cellchat_gene_name)

         name_in_gtf
A1BG            A1BG
A1BG-AS1    A1BG-AS1
A1CF            A1CF
A2M              A2M
A2M-AS1      A2M-AS1
...              ...
ZYG11B        ZYG11B
ZYX              ZYX
ZYXP1          ZYXP1
ZZEF1          ZZEF1
ZZZ3            ZZZ3

[41787 rows x 1 columns]


In [89]:
gene_name_check = pd.read_csv('query_result.csv', index_col=0)

gene_name_check['equal'] = 'N'

for i in gene_name_check.index.tolist():
    
    if i == gene_name_check.loc[i, 'name_in_gtf']:
        
        gene_name_check.loc[i, 'equal'] = 'Y'
        
gene_name_check['equal'].value_counts()

Y    34470
N     7317
Name: equal, dtype: int64

In [90]:
convert_stat = pd.DataFrame(gene_name_check[gene_name_check['equal'] == 'N']['name_in_gtf'].value_counts())

gene_name_check = gene_name_check[gene_name_check['equal'] == 'N']
gene_name_check = gene_name_check[gene_name_check['name_in_gtf'].isin(convert_stat[convert_stat['name_in_gtf'] < 2].index.tolist())]

gene_name_check

,name_in_gtf,equal
AARS1,AARS,N
AARS1P1,AARSP1,N
AAVS1,PPP1R12C,N
ABCF2-H2BE1,ABCF2,N
ABITRAM,FAM206A,N
...,...,...
ZNRD2-AS1,SSSCA1-AS1,N
ZRSR2P1,ZRSR1,N
ZSCAN5DP,ZSCAN5D,N
ZSWIM9,C19orf68,N


In [91]:
len(np.unique(gene_name_check['name_in_gtf']).tolist())

1194

In [92]:
len(np.unique(gene_name_check.index).tolist())

1194

In [93]:
len(gene_name_check)

1194

In [94]:
len(list(set(np.unique(gene_name_check['name_in_gtf']).tolist()).intersection(set(cellchat_gene_name))))

74

In [95]:
len(list(set(np.unique(gene_name_check['name_in_gtf']).tolist()).difference(set(cellchat_gene_name))))

1120

In [96]:
gene_name_check = gene_name_check[gene_name_check['name_in_gtf'].isin(list(set(np.unique(gene_name_check['name_in_gtf']).tolist()).difference(set(cellchat_gene_name))))]
len(list(set(np.unique(gene_name_check['name_in_gtf']).tolist()).intersection(set(cellchat_gene_name))))

0

In [97]:
gene_name_check.to_csv('cellchat_gene_name_replaced.csv')

# DGE

In [3]:
adata = sc.read('/public/home/guogjgroup/ggj/matq_analysis/pan_cancer/all_cell/adata_all_cell_raw_with_anno_latest.h5ad')
all_patient = np.unique(adata.obs['study_id']).tolist()

cancer_adata = sc.read('/public/home/guogjgroup/ggj/matq_analysis/pan_cancer/cancer_module/cnv_high_epi_harmony/cnv_high_epi_harmony_anno_latest.h5ad')

adata = sc.read('/public/home/guogjgroup/ggj/matq_analysis/pan_cancer/all_cell/adata_all_cell_raw_with_anno_latest.h5ad')
adata = adata[adata.obs['lineage'] == 'Epithelium']
adata.obs['leiden'] = adata.obs['Type'].astype(str)
epi_adata = adata.copy()

mono_adata = sc.read('/public/home/guogjgroup/ggj/matq_analysis/pan_cancer/myeloid_cell/adata_all_monocyte_sct_annotation_latest.h5ad')

stroma_adata = sc.read('/public/home/guogjgroup/ggj/matq_analysis/pan_cancer/stromal_cell/adata_all_stromal_cell_sct_annotation_latest.h5ad')

endo_adata = sc.read('/public/home/guogjgroup/ggj/matq_analysis/pan_cancer/endothelial_cell/adata_all_endo_cell_sct_annotation_latest.h5ad')

del adata

/tmp/ipykernel_23703/10841693.py:8: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['leiden'] = adata.obs['Type'].astype(str)


In [4]:
min_cell = 10

for each_patient in all_patient:
    
    each_cca_anno = pd.DataFrame()
    
    for each_lineage in ['Cancer', 'Epi', 'Mono', 'Stroma', 'Endo']:
        
        if each_lineage == 'Cancer':
            
            adata = cancer_adata.copy()
        
        elif each_lineage == 'Epi':
            
            adata = epi_adata.copy()
        
        elif each_lineage == 'Mono':
            
            adata = mono_adata.copy()
            
        elif each_lineage == 'Stroma':
            
            adata = stroma_adata.copy()

        elif each_lineage == 'Endo':
            
            adata = endo_adata.copy()

        adata = adata[adata.obs['study_id'] == each_patient]
        
        if adata.shape[0] > 0:
    
            adata.obs['leiden'] = adata.obs['leiden'].astype(str)
            adata.obs['ccc_anno'] = each_lineage + '(' + adata.obs['leiden'] + ')'

            for each_anno in np.unique(adata.obs['ccc_anno']).tolist():

                if adata[adata.obs['ccc_anno'] == each_anno].shape[0] < min_cell:

                    adata = adata[adata.obs['ccc_anno'] != each_anno]
                    
            if adata.shape[0] > 0:
                
                each_cca_anno = pd.concat([each_cca_anno, adata.obs[['ccc_anno']].copy()], axis=0)
                     
    if each_cca_anno.shape[0] > 0:
        
        adata = sc.read('/public/home/guogjgroup/ggj/matq_analysis/pan_cancer/all_cell/adata_all_cell_raw_with_anno_latest.h5ad')
        adata = adata[each_cca_anno.index.tolist(), :]
        adata.obs['ccc_anno'] = each_cca_anno['ccc_anno']        
        adata.obs[['lineage', 'Type', 'cnv_cluster_type', 'ccc_anno']].to_csv('original_DGE/' + each_patient + '_DGE_for_CellChat.csv')   
        adata.write('original_DGE/' + each_patient + '_DGE_for_CellChat.h5ad')   


/tmp/ipykernel_23703/1293981718.py:42: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['leiden'] = adata.obs['leiden'].astype(str)
/tmp/ipykernel_23703/1293981718.py:42: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['leiden'] = adata.obs['leiden'].astype(str)
/tmp/ipykernel_23703/1293981718.py:42: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['leiden'] = adata.obs['leiden'].astype(str)
/tmp/ipykernel_23703/1293981718.py:42: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['leiden'] = adata.obs['leiden'].astype(str)
/tmp/ipykernel_23703/1293981718.py:59: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['ccc_anno'] = each_cca_anno['ccc_anno']
/tmp/ipykernel_23703/129

/tmp/ipykernel_23703/1293981718.py:42: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['leiden'] = adata.obs['leiden'].astype(str)
/tmp/ipykernel_23703/1293981718.py:59: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['ccc_anno'] = each_cca_anno['ccc_anno']
/tmp/ipykernel_23703/1293981718.py:42: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['leiden'] = adata.obs['leiden'].astype(str)
/tmp/ipykernel_23703/1293981718.py:42: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['leiden'] = adata.obs['leiden'].astype(str)
/tmp/ipykernel_23703/1293981718.py:42: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['leiden'] = adata.obs['leiden'].astype(str)
/tmp/ipykernel_23703/129

# DGE: Gene Name Replacement

In [109]:
for each_patient in [
 'BRCA_p1',
 'BRCA_p2',
 'BRCA_p3',
 'BRCA_p4',
 'COAD_p1',
 'COAD_p2',
 'ESCA_p1',
 'ESCA_p2',
 'ESCA_p3',
 'HCC_p1',
 'HCC_p2',
 'HCC_p3',
 'HCC_p4',
 'HCC_p5',
 'ICC_p1',
 'ICC_p2',
 'ICC_p3',
 'ICC_p4',
 'LUAD_p1',
 'LUAD_p2',
 'LUAD_p3',
 'LUAD_p4',
 'LUAD_p5',
 'LUAD_p6',
 'LUAD_p7',
 'READ_p1',
 'STAD_p1']:
    
    adata = sc.read('original_DGE/' + each_patient + '_DGE_for_CellChat.h5ad')   
        
    for each_cluster in np.unique(adata.obs['ccc_anno']).tolist():

        if adata[adata.obs['ccc_anno'] == each_cluster].shape[0] <= 10:

            adata = adata[adata.obs['ccc_anno'] != each_cluster]
    
    adata_var = pd.DataFrame(adata.var.index).copy()
    adata_var.index = adata_var[0]
    adata_var['name_replaced'] = adata_var.index
    del adata_var[0]

    cellchat_gene_name_replaced = pd.read_csv('cellchat_gene_name_replaced.csv', index_col=0)

    for i in adata_var.index.tolist():

        if i in cellchat_gene_name_replaced['name_in_gtf'].tolist():

            adata_var.loc[i, 'name_replaced'] = ''.join(cellchat_gene_name_replaced[cellchat_gene_name_replaced['name_in_gtf'] == i].index.tolist())
    
    adata = adata[:, adata_var.index.tolist()]
    adata.var.index = adata_var['name_replaced'].tolist()
    
    adata.obs[['lineage', 'Type', 'cnv_cluster_type', 'ccc_anno']].to_csv(each_patient + '_DGE_for_CellChat.csv')   
    adata.write(each_patient + '_DGE_for_CellChat.h5ad')   

